In [1]:
# Set root directory
import os

ROOT_DIR = "D:\Coding\CZ4010\Applied-Cryptography-Project"
os.chdir(ROOT_DIR)

In [2]:
from utils import ascii_to_binary_list, binary_list_to_ascii
from LWE_PKC import LWE_Encrypt, LWE_Decrypt
from math import comb

### 1. Testing out the Cryptosystem


In [3]:
bit_message = [1]

In [4]:
# Weak Parameters
n = 3
q = 17
max_error = 1
S = 2*max_error + 1   # Cardinality of the error set
m = 2*comb(n+S, S)    # The number of eqations must be >> (n+S)C(s)

In [5]:
# Initialize PKC with parameters
lwe_d = LWE_Decrypt(n=n, q=q, max_error=max_error, list_size=m)

# Get public keys
A_list, T_list, q, max_error = lwe_d.get_public_keys()
# print(A_list)
# print(T_list)
print(f"Randomly Initalized Secret: {lwe_d.secret}")

Randomly Initalized Secret: [5, 2, 12]


In [6]:
# Encrypt Message
lwe_e = LWE_Encrypt(A_list, T_list, q, max_error)
A_new, T_send = lwe_e.encrypt_message(bit_message)
print(f"A_new = {A_new}\nT_send = {T_send}")

A_new = [array([10, 10,  1], dtype=int32)]
T_send = [4]


In [7]:
# Decrypt Message
decrypted_binary_message = lwe_d.decrypt_message(A_new, T_send)
decrypted_message = binary_list_to_ascii(decrypted_binary_message)
print(f"Decrypted Message = {decrypted_binary_message}")

Decrypted Message = [1]


### Arora-Ge Attack

An algebraic attack when the max_error parameter is small.

`https://eprint.iacr.org/2014/1018.pdf`


In [60]:
# Weak Parameters
n = 12
q = 1193
max_error = 1
E = 2*max_error + 1   # Cardinality of the error set

# Set the number of samples wisely!
# m = int(2 * comb(n+E, E))   # The number of eqations must be >> (n+S)C(s)
# m = int(2 * n ** (E))
m = 500

In [61]:
# Initialize PKC with parameters
lwe_d = LWE_Decrypt(n=n, q=q, max_error=max_error, list_size=m)

# The secret is randomly initalized in the class
A_list, b_list, q, max_error = lwe_d.get_public_keys()
print(f"Randomly Initalized Secret: {lwe_d.secret}")

Randomly Initalized Secret: [468, 108, 1058, 350, 403, 569, 233, 39, 866, 733, 679, 962]


We are going to recover the secret `s` from `A_list` and `b_list` given the following weak parameters,

1. The error is truncated and takes values in the set $\{-T, ... , -1, 0, 1, ... , T\} \implies |E| = 2T + 1$
   <br>
   <br>
2. The number of equations in `A_list`, $m >> {n + |E| \choose |E|}$


In [62]:
print(f"Cardinality of error set: {E}")
print(f"Number of equations: {len(A_list)} >> {comb(n+E, E)}")

Cardinality of error set: 3
Number of equations: 500 >> 455


In [63]:
from itertools import product
from collections import defaultdict
from sympy import symbols, Matrix, GF
from sympy.polys.matrices import DomainMatrix

In [64]:
# Define the secret vector
secret_vector = symbols(f'x1:{n+1}')
secret_vector

(x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12)

In [65]:
polynomials_over_Zq = []
error_set = [i for i in range(-max_error, max_error+1)] 

# Begin constructing the polynomials for each LWE instance <A, b>
for A, b in zip(A_list, b_list):
    # Initalize the polynomial term to the identity polynomial of the finite field
    polynomial_over_Zq = GF(q)[secret_vector](1)

    for e in error_set:
        # Multiply each variable by its corresponding weight
        weighted_polynomial = sum(w * var for w, var in zip(A, secret_vector))
        # print(weighted_polynomial)

        # Construct the weighted polynomial (this is the AT*s term in the equation)
        weighted_secret_polynomial_over_Zq = GF(q)[secret_vector](weighted_polynomial)
        # print(weighted_secret_polynomial_over_Zq)

        # Complete the term (b - AT*s - e)
        term = b - weighted_secret_polynomial_over_Zq - e
        # print(term)

        # Accumulate the product of each term over Zq
        polynomial_over_Zq = polynomial_over_Zq * term
        # print(polynomial_over_Zq)
    
    # Collect the polynomials
    polynomials_over_Zq.append(polynomial_over_Zq)

# Sanity check
assert len(polynomials_over_Zq) == len(A_list)

In [66]:
def generate_tuples(n, d):
    # Use itertools.product to generate all tuples
    tuples = list(product(range(d + 1), repeat=n))
    
    # Remove tuples that have a degree > d
    tuples_pruned = [x for x in tuples if sum(x) <= d]

    return tuples_pruned

In [67]:
coefficients_dicts = []
tuples = generate_tuples(n, E)

for polynomial in polynomials_over_Zq:
    coefficients_dict = defaultdict(int, {key: 0 for key in tuples})

    for term_key, coeff in polynomial.terms():
        coefficients_dict[term_key] = int(coeff) # NOTE: Converting to negative int!!!

    coefficients_dicts.append(coefficients_dict)

In [68]:
# # Checking an example
# coefficients_dicts[0]

In [69]:
row_order = []

# Let's keep the degree 1 terms up front for convenince
for i in range(n):
    term = [0]*n
    term[i] = 1
    row_order.append(tuple(term))

# We want the secret at the start and the constant term at the end
seen = set(row_order)
constant_term_key = tuple([0]*n)
candidates = generate_tuples(n, E)

for x in candidates:
    if x not in seen and x != constant_term_key:
        row_order.append(x)

# Ensure the constant term is at the end
row_order.append(constant_term_key)

# Sanity check
assert len(row_order) == len(candidates)

In [70]:
coefficient_matrix = []
rhs = []

for coeff_dict in coefficients_dicts:
    row = []

    for key in row_order[:-1]:
        row.append(coeff_dict[key])
    
    # Append row
    coefficient_matrix.append(row)

    # Append the rhs
    negative_constant_term = -coeff_dict[row_order[-1]]
    rhs.append(negative_constant_term)

In [71]:
# Solving linear system using DomainMatrix
m = Matrix(coefficient_matrix)
b = Matrix(rhs)

# Convert matrices to finite field of order q (q is prime):
K = GF(q, symmetric=False)
dm = DomainMatrix.from_Matrix(m).convert_to(K)
bm = DomainMatrix.from_Matrix(b).convert_to(K)

# Print shape of system of equations
print(dm.shape)
print(bm.shape)

# Solve and convert back to an ordinary Matrix:
solution_vector = dm.lu_solve(bm).to_Matrix()
print(solution_vector)

(500, 454)
(500, 1)
Matrix([[468], [108], [1058], [350], [403], [569], [233], [39], [866], [733], [679], [962], [869], [878], [627], [709], [543], [1025], [60], [83], [1108], [226], [286], [750], [439], [1189], [1024], [870], [378], [964], [1058], [167], [196], [102], [298], [64], [800], [752], [466], [4], [50], [1047], [535], [487], [235], [593], [896], [1148], [851], [463], [419], [370], [426], [700], [399], [696], [328], [584], [814], [631], [114], [862], [1055], [860], [731], [545], [61], [190], [251], [166], [882], [161], [985], [756], [1099], [1038], [736], [583], [1070], [252], [314], [72], [604], [57], [917], [129], [530], [889], [1151], [984], [559], [1012], [56], [1173], [720], [700], [943], [454], [45], [342], [730], [774], [794], [717], [200], [99], [641], [562], [524], [154], [216], [775], [740], [941], [41], [92], [458], [379], [802], [481], [552], [1160], [537], [528], [1154], [658], [440], [958], [510], [728], [45], [410], [353], [642], [823], [473], [544], [34], [208],

In [72]:
print(f"Randomly Initalized Secret:\t\t{lwe_d.secret}")
print(f"Secret obtained from Arora-Ge Attack:\t{solution_vector[:n]}")

Randomly Initalized Secret:		[468, 108, 1058, 350, 403, 569, 233, 39, 866, 733, 679, 962]
Secret obtained from Arora-Ge Attack:	[468, 108, 1058, 350, 403, 569, 233, 39, 866, 733, 679, 962]


In [76]:
print(f"Did we correctly determine the secret? \n{lwe_d.secret == solution_vector[:n]}")

Did we correctly determine the secret? 
True


Works pretty well for `n <= 10` and `max_error = 1 or 2`. Basically ensure `m <= 1000` else it takes VERY long to run. 


### Efficient Implementation using Groebner Basis

Does not work very well, better implement in Sage.


In [73]:
# from sympy import symbols, Poly, GF, groebner

# def arora_ge_attack(q, A, b, E, S=None):
#     """
#     Recovers the secret key s from the LWE samples A and b.
#     More information: "The Learning with Errors Problem: Algorithms" (Section 1)
#     :param q: the modulus
#     :param A: the matrix A, represented as a list of lists
#     :param b: the vector b, represented as a list
#     :param E: the possible error values
#     :param S: the possible values of the entries in s (default: None)
#     :return: a list representing the secret key s
#     """
#     m = len(A)
#     n = len(A[0])

#     x = symbols(tuple(f"x{i}" for i in range(n)))
#     gf = GF(q)
#     pr = gf[x]
#     gens = pr.symbols

#     f = []
#     for i in range(m):
#         p = 1
#         for e in E:
#             p *= (b[i] - sum(A[i][j] * gens[j] for j in range(n)) - e)
#         f.append(p)

#     if S is not None:
#         # Use information about the possible values for s to add more polynomials.
#         for j in range(n):
#             p = 1
#             for s in S:
#                 p *= (gens[j] - s)
#             f.append(p)

#     ideal = [Poly(poly, gens) for poly in f]
#     basis = groebner(ideal, gens, order='lex')

#     s = []
#     for poly in basis:
#         #assert poly.variables() == 1 and poly.degree() == 1
#         s.append(int(-poly.coeffs()[0]))

#     return s

In [74]:
# error_values = [i for i in range(-max_error, max_error+1)] 
# x = arora_ge_attack(q=q, A=A_list, b=b_list, E=error_values)